<br>


# Answering Business Questions: Intermediate SQL


<br>

This is my attempt at Dataquest's Guided Project: 'Answering Business Questions using SQL' -->
https://www.dataquest.io/m/191-guided-project-answering-business-questions-using-sql/

----

    
    This project uses the chinook database, which contains information about albums, artists, sales, and    
    employees. I will be following along with the guided project, which offers mock business questions that     can be answered by analyzing information from this database using more complex sql techniques.
    
    
-----

#### Load sql and connect to database

In [1]:
%%capture
%load_ext sql

In [2]:
%sql sqlite:///chinook.db

'Connected: @chinook.db'

### Explore tables

In [132]:
%%sql
SELECT name, type
    FROM sqlite_master
    WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [7]:
%%sql
SELECT *
    FROM album
    LIMIT 5;

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [28]:
%%sql
SELECT *
    FROM artist
    WHERE name LIKE "AC%"
    LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
214,Academy of St. Martin in the Fields & Sir Neville Marriner
215,Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner
222,"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair"


In [20]:
%%sql
SELECT al.title 'album title', ar.name artist
    FROM album al
    INNER JOIN artist ar on ar.artist_id = al.artist_id
    LIMIT 5;

 * sqlite:///chinook.db
Done.


album title,artist
For Those About To Rock We Salute You,AC/DC
Balls to the Wall,Accept
Restless and Wild,Accept
Let There Be Rock,AC/DC
Big Ones,Aerosmith


---------

## 1. Which genres sell the most tracks in the USA?

**Get the amount of sales per track**

In [59]:
%%sql
SELECT SUM(quantity) total_sales, track_id
    FROM invoice_line
    GROUP BY track_id
    ORDER BY 1 DESC
    LIMIT 1;

 * sqlite:///chinook.db
Done.


total_sales,track_id
31,3336


**Get only invoices from customers in the USA**

In [58]:
%%sql
SELECT invoice_id
    FROM invoice
    WHERE billing_country = "USA"
    LIMIT 1;

 * sqlite:///chinook.db
Done.


invoice_id
1


**Get total amount of sales per track in the USA (combine first two queries)**

In [80]:
%%sql
SELECT SUM(il.quantity) total_sales, il.track_id
    FROM invoice_line il
    
    INNER JOIN (
            SELECT invoice_id
                FROM invoice
                WHERE billing_country = "USA"
    ) inv ON inv.invoice_id = il.invoice_id
    
    GROUP BY il.track_id
    ORDER BY 1 DESC
    LIMIT 1;

 * sqlite:///chinook.db
Done.


total_sales,track_id
6,3336


**Now use the sub-query above to get the genre from the genre table by first joining to the track table, then accessing the genre by genre_id**

In [94]:
%%sql
SELECT SUM(il.quantity) total_sales, g.name genre
    FROM invoice_line il
    
    INNER JOIN (
            SELECT invoice_id
                FROM invoice
                WHERE billing_country = "USA"
    ) inv ON inv.invoice_id = il.invoice_id

    INNER JOIN track t ON t.track_id = il.track_id
    
    INNER JOIN genre g ON g.genre_id = t.genre_id

    GROUP BY g.name
    ORDER BY 1 DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


total_sales,genre
561,Rock
130,Alternative & Punk
124,Metal
53,R&B/Soul
36,Blues
35,Alternative
22,Pop
22,Latin
20,Hip Hop/Rap
14,Jazz


**Above is a list of the total track sales by genre ranked in descending order**

### Business task: 

Tasked with selecting the first ***three*** albums that will be added to a music store from a list. The list contains the artist names and the genre of music they produce. 

    Regal                 Hip-Hop
    Red Tone              Punk
    Meteor and the Girls  Pop
    Slim Jim Bites        Blues

### My recommendation:
    
I would say that, given the sales data pulled from the database, the store should go with all the albums except the hip hop album if their goal is to increase the likelihood that the new records will be purchased. This is based solely on genre sales and the corresponding genres of the options given.

--------

## 2. Are any of the employees performing better or worse than the others?  
#### (Based on purchases of customers belonging to sales support agents)

In [123]:
%%sql
SELECT e.employee_id, e.last_name, e.title, e.hire_date, 
        e.birthdate, SUM(i.total) total_sales, COUNT(c.support_rep_id) no_sales, 
        MAX(i.total) largest_sale, AVG(i.total) avg_invoice_amt
    
    FROM employee e
    
    INNER JOIN customer c ON c.support_rep_id = e.employee_id
    
    INNER JOIN invoice i ON i.customer_id = c.customer_id
   
    GROUP BY e.employee_id
    ORDER BY 6 DESC;

 * sqlite:///chinook.db
Done.


employee_id,last_name,title,hire_date,birthdate,total_sales,no_sales,largest_sale,avg_invoice_amt
3,Peacock,Sales Support Agent,2017-04-01 00:00:00,1973-08-29 00:00:00,1731.5100000000039,212,23.759999999999998,8.167500000000018
4,Park,Sales Support Agent,2017-05-03 00:00:00,1947-09-19 00:00:00,1584.0000000000034,214,19.8,7.401869158878521
5,Johnson,Sales Support Agent,2017-10-17 00:00:00,1965-03-03 00:00:00,1393.920000000002,188,16.83,7.414468085106393


### My analysis of customer performance

From the data above, it appears there is some performance difference between the Sales Support Agents. 
Peacock, having roughly the same amount of sales as Park, has a higher average invoice amount. Caution must be taken in using this as a metric because Peacock does have an outlier sale of almost $24.

Nonetheless, Peacock has the greatest total_sales amount by almost $150. Having two less sales than Park, the difference in how long they have been with the company doesn't account for their total_sales difference.

Johnson has about the same average invoice amount as Park, which is a better indicator of Johnson's performace as they have been with the company a shorter amount of time. 

Overall, **Peacock is the best-performing Sales Support Agent**. Park and Johnson are performing at a similar level to each other. 

----------

## 3. Analysis of Sales by Country

### Task: 

For each country in the dataset, calculate:
    1. Total number of customers
    2. Total value of sales
    3. Average value of sales per customer       

### Breakdown of countries and customers

<br>(First create view to narrow down necessary info)

In [128]:
%%sql
CREATE VIEW customers_by_country AS
    SELECT country, COUNT(customer_id) no_customers
        FROM customer
        GROUP BY country
        ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


[]

In [158]:
%%sql
SELECT *
    FROM customers_by_country
    LIMIT 5;

 * sqlite:///chinook.db
Done.


country,no_customers
USA,13
Canada,8
France,5
Brazil,5
Germany,4


**Get total sales by country**

In [170]:
%%sql
SELECT billing_country country, ROUND(SUM(total), 2) total_sales
    FROM invoice
    GROUP BY billing_country
    ORDER BY 2 DESC
    LIMIT 5;

 * sqlite:///chinook.db
Done.


country,total_sales
USA,1040.49
Canada,535.59
Brazil,427.68
France,389.07
Germany,334.62


In [178]:
%%sql
SELECT cbc.*, invoice.total_sales, ROUND(invoice.avg_sale, 2) avg_sale_per_customer
    FROM customers_by_country cbc
    
    INNER JOIN (
            SELECT billing_country, ROUND(SUM(total), 2) total_sales, AVG(total) avg_sale
                FROM invoice
                GROUP BY billing_country
    )invoice ON cbc.country = invoice.billing_country
    
    ORDER BY cbc.no_customers DESC;

 * sqlite:///chinook.db
Done.


country,no_customers,total_sales,avg_sale_per_customer
USA,13,1040.49,7.94
Canada,8,535.59,7.05
France,5,389.07,7.78
Brazil,5,427.68,7.01
Germany,4,334.62,8.16
United Kingdom,3,245.52,8.77
Portugal,2,185.13,6.38
India,2,183.15,8.72
Czech Republic,2,273.24,9.11
Sweden,1,75.24,7.52


Breaking down the data above, we can see that the USA has the most customers and the highest total sales. 